In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import re  #regular expression

In [2]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q17-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q17-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q17-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q17-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [3]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 83 columns):
 #   Column                                                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                                                      --------------  -----  
 0   Response ID                                                                                                                                                 396 non-null    int64  
 1   Large Roadside Posters/ billboards:BOSCH:GM -  And which of these types of outdoor advertising have you seen recently for these brands?                     9 non-null      float64
 2   On bus-stops, streetlamps or retailer carparks:BOSCH:GM -  And which of these types of outdoor advertising have you seen recently for these brands?   

In [4]:
Germany.head(2) #initial dataset

,Response ID,Large Roadside Posters/ billboards:BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:BOSCH:GM - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:MIELE:GM - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:MIELE:GM - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):HAIER:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):HAIER:GM - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:HAIER:GM - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:ROWENTA:GM - And which of these types of outdoor advertising have you seen recently for these brands?
0,78230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_GM['Offline_source_cleaned'] = unpivoted_GM['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_GM = unpivoted_GM[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_GM['Offline_source_cleaned'] = unpivoted_GM['Offline_source_cleaned'].str.replace(
    ":GM -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_GM.head())

   Response ID                                     Offline_source  \
0        78230  Large Roadside Posters/ billboards:BOSCH:GM - ...   
1        78263  Large Roadside Posters/ billboards:BOSCH:GM - ...   
2        78268  Large Roadside Posters/ billboards:BOSCH:GM - ...   
3        78278  Large Roadside Posters/ billboards:BOSCH:GM - ...   
4        78299  Large Roadside Posters/ billboards:BOSCH:GM - ...   

                     Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:BOSCH  NaN  
1  Large Roadside Posters/ billboards:BOSCH  NaN  
2  Large Roadside Posters/ billboards:BOSCH  NaN  
3  Large Roadside Posters/ billboards:BOSCH  NaN  
4  Large Roadside Posters/ billboards:BOSCH  NaN  


In [6]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_GM['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [7]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_GM = unpivoted_GM.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_GM = unpivoted_GM[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_GM['Y/N'] = np.where(unpivoted_GM['Y/N'].isna(), 0, unpivoted_GM['Y/N'])

In [8]:
unpivoted_GM.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78230,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
1,78263,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
2,78268,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
3,78278,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
4,78299,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH


In [9]:
count = unpivoted_GM['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32019
2.0       107
1.0       104
3.0        64
4.0        61
6.0        47
5.0        46
99.0       24
Name: count, dtype: int64


Total number of rows: 32472


### Sweden

In [10]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 80 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  400 non-null    int64  
 1   Large Roadside Posters/ billboards:ELECTROLUX:SW -  And which of these types of outdoor advertising have you seen recently for these brands?                 6 non-null      float64
 2   On bus-stops, streetlamps or retailer carparks:ELECTROLUX:SW -  And which of these types of outdoor advertising have you seen recently for these b

In [11]:
Sweden.head(2) #initial dataset

,Response ID,Large Roadside Posters/ billboards:ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:ELECTROLUX:SW - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:BOSCH:SW - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:BOSCH:SW - And which of these types of outdoor advertising have you seen recently for these brands?",...,Large Roadside Posters/ billboards:AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:AEG:SW - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?",At underground / metro stations:ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:ROBOROCK:SW - And which of these types of outdoor advertising have you seen recently for these brands?
0,78236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_SW['Offline_source_cleaned'] = unpivoted_SW['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_SW = unpivoted_SW[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_SW['Offline_source_cleaned'] = unpivoted_SW['Offline_source_cleaned'].str.replace(
    ":SW -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_SW.head())

   Response ID                                     Offline_source  \
0        78236  Large Roadside Posters/ billboards:ELECTROLUX:...   
1        78238  Large Roadside Posters/ billboards:ELECTROLUX:...   
2        78241  Large Roadside Posters/ billboards:ELECTROLUX:...   
3        78246  Large Roadside Posters/ billboards:ELECTROLUX:...   
4        78248  Large Roadside Posters/ billboards:ELECTROLUX:...   

                          Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:ELECTROLUX  NaN  
1  Large Roadside Posters/ billboards:ELECTROLUX  NaN  
2  Large Roadside Posters/ billboards:ELECTROLUX  NaN  
3  Large Roadside Posters/ billboards:ELECTROLUX  NaN  
4  Large Roadside Posters/ billboards:ELECTROLUX  NaN  


In [13]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_SW['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [14]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_SW = unpivoted_SW.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_SW = unpivoted_SW[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_SW['Y/N'] = np.where(unpivoted_SW['Y/N'].isna(), 0, unpivoted_SW['Y/N'])

In [15]:
unpivoted_SW.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78236,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX
1,78238,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX
2,78241,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX
3,78246,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX
4,78248,Large Roadside Posters/ billboards:ELECTROLUX:...,Large Roadside Posters/ billboards,0.0,ELECTROLUX


In [16]:
count = unpivoted_SW['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")


Y/N
0.0     31216
2.0        76
5.0        71
3.0        63
1.0        58
4.0        52
6.0        45
99.0       19
Name: count, dtype: int64


Total number of rows: 31600


### Italy

In [17]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 85 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  398 non-null    int64  
 1   Large Roadside Posters/ billboards:SAMSUNG:IT -  And which of these types of outdoor advertising have you seen recently for these brands?                    18 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:IT -  And which of these types of outdoor advertising have you seen recently for these bran

In [18]:
Italy.head(2) #initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:IT - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:BOSCH:IT - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:BOSCH:IT - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):HAIER:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):HAIER:IT - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:HAIER:IT - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SIEMENS:IT - And which of these types of outdoor advertising have you seen recently for these brands?
0,78233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_IT['Offline_source_cleaned'] = unpivoted_IT['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_IT = unpivoted_IT[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_IT['Offline_source_cleaned'] = unpivoted_IT['Offline_source_cleaned'].str.replace(
    ":IT -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_IT.head())

   Response ID                                     Offline_source  \
0        78233  Large Roadside Posters/ billboards:SAMSUNG:IT ...   
1        78237  Large Roadside Posters/ billboards:SAMSUNG:IT ...   
2        78242  Large Roadside Posters/ billboards:SAMSUNG:IT ...   
3        78300  Large Roadside Posters/ billboards:SAMSUNG:IT ...   
4        78315  Large Roadside Posters/ billboards:SAMSUNG:IT ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [20]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_IT['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [21]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_IT = unpivoted_IT.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_IT = unpivoted_IT[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_IT['Y/N'] = np.where(unpivoted_IT['Y/N'].isna(), 0, unpivoted_IT['Y/N'])

In [22]:
unpivoted_IT.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78233,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,78237,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,78242,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,78300,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,78315,Large Roadside Posters/ billboards:SAMSUNG:IT ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


In [23]:
count = unpivoted_IT['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32894
2.0        93
1.0        88
3.0        83
5.0        82
6.0        76
4.0        75
99.0       41
Name: count, dtype: int64


Total number of rows: 33432


### Australia

In [24]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 85 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     397 non-null    int64  
 1   Large Roadside Posters/ billboards:SAMSUNG:AU -  And which of these types of outdoor advertising have you seen recently for these brands?                       8 non-null      float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:AU -  And which of these types of outdoor advertising have you seen recently fo

In [25]:
Australia.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:LG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:LG:AU - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):HAIER:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):HAIER:AU - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:HAIER:AU - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:AEG:AU - And which of these types of outdoor advertising have you seen recently for these brands?
0,78293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_AU['Offline_source_cleaned'] = unpivoted_AU['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_AU = unpivoted_AU[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_AU['Offline_source_cleaned'] = unpivoted_AU['Offline_source_cleaned'].str.replace(
    ":AU -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_AU.head())

   Response ID                                     Offline_source  \
0        78293  Large Roadside Posters/ billboards:SAMSUNG:AU ...   
1        78361  Large Roadside Posters/ billboards:SAMSUNG:AU ...   
2        78394  Large Roadside Posters/ billboards:SAMSUNG:AU ...   
3        78406  Large Roadside Posters/ billboards:SAMSUNG:AU ...   
4        78429  Large Roadside Posters/ billboards:SAMSUNG:AU ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [27]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_AU['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [28]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_AU = unpivoted_AU.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_AU = unpivoted_AU[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_AU['Y/N'] = np.where(unpivoted_AU['Y/N'].isna(), 0, unpivoted_AU['Y/N'])

In [29]:
unpivoted_AU.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78293,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,78361,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,78394,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,78406,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,78429,Large Roadside Posters/ billboards:SAMSUNG:AU ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


In [30]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG         2779
LG              2779
DYSON           2779
WESTINGHOUSE    2779
BOSCH           2779
HISENSE         2779
MIELE           2779
SMEG            2779
ELECTROLUX      2779
HAIER           2779
AEG             2779
Name: count, dtype: int64


Total number of rows: 33348


In [31]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [32]:
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand:", na_count)

Number of NaN values in Brand: 0


### UK

In [33]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 82 columns):
 #   Column                                                                                                                                                     Non-Null Count  Dtype  
---  ------                                                                                                                                                     --------------  -----  
 0   Response ID                                                                                                                                                396 non-null    int64  
 1   Large Roadside Posters/ billboards:SAMSUNG:UK -  And which of these types of outdoor advertising have you seen recently for these brands?                  14 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:UK -  And which of these types of outdoor advertising have you seen recently for these brands?     

In [34]:
UK.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:LG:UK - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:LG:UK - And which of these types of outdoor advertising have you seen recently for these brands?",...,On transport (eg bus-sides and taxis):SIEMENS:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SIEMENS:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SIEMENS:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SIEMENS:UK - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):NEFF:UK - And which of these types of outdoor advertising have you seen recently for these brands?
0,78251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_UK['Offline_source_cleaned'] = unpivoted_UK['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_UK = unpivoted_UK[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_UK['Offline_source_cleaned'] = unpivoted_UK['Offline_source_cleaned'].str.replace(
    ":UK -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_UK.head())

   Response ID                                     Offline_source  \
0        78251  Large Roadside Posters/ billboards:SAMSUNG:UK ...   
1        78258  Large Roadside Posters/ billboards:SAMSUNG:UK ...   
2        78259  Large Roadside Posters/ billboards:SAMSUNG:UK ...   
3        78261  Large Roadside Posters/ billboards:SAMSUNG:UK ...   
4        78262  Large Roadside Posters/ billboards:SAMSUNG:UK ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [36]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_UK['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [37]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_UK = unpivoted_UK.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_UK = unpivoted_UK[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_UK['Y/N'] = np.where(unpivoted_UK['Y/N'].isna(), 0, unpivoted_UK['Y/N'])

In [38]:
unpivoted_UK.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78251,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,78258,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,78259,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,78261,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,78262,Large Roadside Posters/ billboards:SAMSUNG:UK ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


In [39]:
count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG     2772
LG          2772
HOTPOINT    2772
BOSCH       2772
DYSON       2772
BEKO        2772
ZANUSSI     2772
MIELE       2772
AEG         2772
SHARK       2376
SIEMENS     2376
NEFF        2376
Name: count, dtype: int64


Total number of rows: 32076


### Poland

In [40]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 82 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  400 non-null    int64  
 1   Large Roadside Posters/ billboards:SAMSUNG:PO -  And which of these types of outdoor advertising have you seen recently for these brands?                    20 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:PO -  And which of these types of outdoor advertising have you seen recently for these bran

In [41]:
Poland.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:BOSCH:PO - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:BOSCH:PO - And which of these types of outdoor advertising have you seen recently for these brands?",...,On transport (eg bus-sides and taxis):MIELE:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:MIELE:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):MIELE:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):MIELE:PO - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:AEG:PO - And which of these types of outdoor advertising have you seen recently for these brands?
0,78295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_PO['Offline_source_cleaned'] = unpivoted_PO['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_PO = unpivoted_PO[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_PO['Offline_source_cleaned'] = unpivoted_PO['Offline_source_cleaned'].str.replace(
    ":PO -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_PO.head())

   Response ID                                     Offline_source  \
0        78295  Large Roadside Posters/ billboards:SAMSUNG:PO ...   
1        78297  Large Roadside Posters/ billboards:SAMSUNG:PO ...   
2        78309  Large Roadside Posters/ billboards:SAMSUNG:PO ...   
3        78317  Large Roadside Posters/ billboards:SAMSUNG:PO ...   
4        78344  Large Roadside Posters/ billboards:SAMSUNG:PO ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [43]:
# import re

# # Initialize lists to store processed data
# brand_names = []
# ad_types = []

# # Process each item in 'Offline_source_cleaned'
# for entry in unpivoted_PO['Offline_source_cleaned']:
#     # Ensure entry is a string, replace NaN/float values with empty string
#     entry = str(entry) if isinstance(entry, str) else ""

#     # Extract brand name (e.g., Samsung, LG, etc.)
#     brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
#     if brand:
#         brand_name = brand.group(1)
#     else:
#         brand_name = None
    
#     # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
#     ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
#     if ad_type:
#         ad_type_name = ad_type.group(0).strip()
#     else:
#         ad_type_name = None

#     # Append to lists
#     brand_names.append(brand_name)
#     ad_types.append(ad_type_name)

# # Add extracted information to the DataFrame
# unpivoted_PO['Brand'] = brand_names
# unpivoted_PO['Ad_Type'] = ad_types


In [44]:
# import re

# # Initialize lists to store processed data
# brand_names = []
# ad_types = []

# # Process each item in 'Offline_source_cleaned'
# for entry in unpivoted_PO['Offline_source_cleaned']:
#     # Convert entry to a string to avoid TypeError
#     entry_str = str(entry)
    
#     # Extract brand name (e.g., Samsung, LG, etc.)
#     brand = re.search(r":([A-Za-z]+)$", entry_str)  # Adjusted to capture brand names at the end of the string
#     if brand:
#         brand_name = brand.group(1)
#     else:
#         brand_name = None
    
#     # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
#     ad_type = re.search(r"^[^:]+", entry_str)  # Capture everything before the first colon
#     if ad_type:
#         ad_type_name = ad_type.group(0).strip()
#     else:
#         ad_type_name = None

#     # Append to lists
#     brand_names.append(brand_name)
#     ad_types.append(ad_type_name)

# # Add extracted information to the DataFrame
# unpivoted_PO['Brand'] = brand_names
# unpivoted_PO['Ad_Type'] = ad_types


In [45]:
import re

# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_PO['Offline_source_cleaned']:
    # Convert entry to a string to avoid TypeError
    entry_str = str(entry)
    
    # Debugging print to check the entry format
    print("Processing entry:", entry_str)
    
    # Extract brand name (e.g., Samsung, LG, etc.)
    # Adjust regex if brand names are separated by spaces or other characters
    brand = re.search(r":\s*([A-Za-z]+)$", entry_str)  # Allows optional whitespace after the colon
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry_str)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

# Optional: Check the resulting DataFrame
print(unpivoted_PO[['Offline_source_cleaned', 'Brand', 'Ad_Type']])


Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large Roadside Posters/ billboards:SAMSUNG
Processing entry: Large 

In [46]:
# # Initialize lists to store processed data
# brand_names = []
# ad_types = []

# # Process each item in 'Offline_source_cleaned'
# for entry in unpivoted_PO['Offline_source_cleaned']:
#     # Extract brand name (e.g., Samsung, LG, etc.)
#     brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
#     if brand:
#         brand_name = brand.group(1)
#     else:
#         brand_name = None
    
#     # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
#     ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
#     if ad_type:
#         ad_type_name = ad_type.group(0).strip()
#     else:
#         ad_type_name = None

#     # Append to lists
#     brand_names.append(brand_name)
#     ad_types.append(ad_type_name)

# # Add extracted information to the DataFrame
# unpivoted_PO['Brand'] = brand_names
# unpivoted_PO['Ad_Type'] = ad_types

In [47]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_PO = unpivoted_PO.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_PO = unpivoted_PO[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_PO['Y/N'] = np.where(unpivoted_PO['Y/N'].isna(), 0, unpivoted_PO['Y/N'])

In [48]:
count = unpivoted_PO['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     31758
2.0       141
3.0       133
1.0       131
5.0        88
6.0        80
4.0        44
99.0       25
Name: count, dtype: int64


Total number of rows: 32400


In [49]:
unpivoted_PO.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78295,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,78297,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,78309,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,78317,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,78344,Large Roadside Posters/ billboards:SAMSUNG:PO ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


### Vietnam

In [50]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 77 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  400 non-null    int64  
 1   Large Roadside Posters/ billboards:SAMSUNG:VN -  And which of these types of outdoor advertising have you seen recently for these brands?                    85 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:SAMSUNG:VN -  And which of these types of outdoor advertising have you seen recently for these bran

In [51]:
Vietnam.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:VN - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:PANASONIC:VN - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:PANASONIC:VN - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):PANASONIC:VN - And which of these types of outdoor advertising have you seen recently for these brands?,...,On transport (eg bus-sides and taxis):BOSCH:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:BOSCH:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):BOSCH:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):BOSCH:VN - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):MIDEA:VN - And which of these types of outdoor advertising have you seen recently for these brands?
0,78271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_VN['Offline_source_cleaned'] = unpivoted_VN['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_VN = unpivoted_VN[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_VN['Offline_source_cleaned'] = unpivoted_VN['Offline_source_cleaned'].str.replace(
    ":VN -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_VN.head())

   Response ID                                     Offline_source  \
0        78271  Large Roadside Posters/ billboards:SAMSUNG:VN ...   
1        78275  Large Roadside Posters/ billboards:SAMSUNG:VN ...   
2        78287  Large Roadside Posters/ billboards:SAMSUNG:VN ...   
3        78291  Large Roadside Posters/ billboards:SAMSUNG:VN ...   
4        78358  Large Roadside Posters/ billboards:SAMSUNG:VN ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  NaN  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  NaN  
4  Large Roadside Posters/ billboards:SAMSUNG  1.0  


In [53]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_VN['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [54]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_VN = unpivoted_VN.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_VN = unpivoted_VN[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_VN['Y/N'] = np.where(unpivoted_VN['Y/N'].isna(), 0, unpivoted_VN['Y/N'])

In [55]:
unpivoted_VN.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78271,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,78275,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
2,78287,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,78291,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
4,78358,Large Roadside Posters/ billboards:SAMSUNG:VN ...,Large Roadside Posters/ billboards,1.0,SAMSUNG


In [56]:
count = unpivoted_VN['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     27828
1.0       561
2.0       476
5.0       469
3.0       432
6.0       373
4.0       256
99.0        5
Name: count, dtype: int64


Total number of rows: 30400


### Switzerland

In [57]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 84 columns):
 #   Column                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                       --------------  -----  
 0   Response ID                                                                                                                                                  398 non-null    int64  
 1   Large Roadside Posters/ billboards:V ZUG:SZ -  And which of these types of outdoor advertising have you seen recently for these brands?                      12 non-null     float64
 2   On bus-stops, streetlamps or retailer carparks:V ZUG:SZ -  And which of these types of outdoor advertising have you seen recently for these brands

In [58]:
Switzerland.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:V ZUG:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:ELECTROLUX:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:ELECTROLUX:SZ - And which of these types of outdoor advertising have you seen recently for these brands?",...,At sports stadiums (e.g. on the perimeter boards):LIEBHERR:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):LIEBHERR:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:LIEBHERR:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SCHULTHESS:SZ - And which of these types of outdoor advertising have you seen recently for these brands?
0,78375,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0
1,78387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_SZ['Offline_source_cleaned'] = unpivoted_SZ['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_SZ['Offline_source_cleaned'] = unpivoted_SZ['Offline_source_cleaned'].str.replace(
    ":SZ -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_SZ.head())

   Response ID                                     Offline_source  \
0        78375  Large Roadside Posters/ billboards:V ZUG:SZ - ...   
1        78387  Large Roadside Posters/ billboards:V ZUG:SZ - ...   
2        78581  Large Roadside Posters/ billboards:V ZUG:SZ - ...   
3        78735  Large Roadside Posters/ billboards:V ZUG:SZ - ...   
4        78849  Large Roadside Posters/ billboards:V ZUG:SZ - ...   

                     Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:V ZUG  1.0  
1  Large Roadside Posters/ billboards:V ZUG  NaN  
2  Large Roadside Posters/ billboards:V ZUG  NaN  
3  Large Roadside Posters/ billboards:V ZUG  NaN  
4  Large Roadside Posters/ billboards:V ZUG  NaN  


In [60]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_SZ['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [61]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_SZ['Y/N'] = np.where(unpivoted_SZ['Y/N'].isna(), 0, unpivoted_SZ['Y/N'])

In [62]:
unpivoted_SZ.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78375,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,1.0,None
1,78387,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None
2,78581,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None
3,78735,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None
4,78849,Large Roadside Posters/ billboards:V ZUG:SZ - ...,Large Roadside Posters/ billboards,0.0,None


In [63]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
ELECTROLUX    2786
MIELE         2786
DYSON         2786
BOSCH         2786
SAMSUNG       2786
SIEMENS       2786
PHILIPS       2786
AEG           2786
LIEBHERR      2786
SCHULTHESS    2786
BAUKNECHT     2388
Name: count, dtype: int64


Total number of rows: 33034


In [64]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)


Number of NaN values in Brand after replacement: 0


In [65]:
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand:", na_count)

Number of NaN values in Brand: 0


### Thailand

In [66]:
Thailand.info

<bound method DataFrame.info of      Response ID  \
0          78252   
1          78269   
2          78313   
3          78345   
4          78363   
..           ...   
394        95265   
395        95350   
396        95488   
397        95492   
398        95815   

     Large Roadside Posters/ billboards:SAMSUNG:TH -  And which of these types of outdoor advertising have you seen recently for these brands?  \
0                                                  NaN                                                                                           
1                                                  1.0                                                                                           
2                                                  NaN                                                                                           
3                                                  1.0                                                                                         

In [67]:
Thailand.head(2) # initial dataset

,Response ID,Large Roadside Posters/ billboards:SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:SAMSUNG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:LG:TH - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:LG:TH - And which of these types of outdoor advertising have you seen recently for these brands?",...,At underground / metro stations:BOSCH:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):BOSCH:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):BOSCH:TH - And which of these types of outdoor advertising have you seen recently for these brands?,None of the above:BOSCH:TH - And which of these types of outdoor advertising have you seen recently for these brands?,Large Roadside Posters/ billboards:DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,"On bus-stops, streetlamps or retailer carparks:DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?",On transport (eg bus-sides and taxis):DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At underground / metro stations:DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At sports stadiums (e.g. on the perimeter boards):DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?,At events (eg concerts / festivals):DYSON:TH - And which of these types of outdoor advertising have you seen recently for these brands?
0,78252,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78269,1.0,2.0,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Unpivot the DataFrame, renaming 'Variable' to 'Offline_source'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='Offline_source', value_name='Y/N')

# Create a duplicate of the 'Offline_source' column named 'Offline_source_cleaned'
unpivoted_TH['Offline_source_cleaned'] = unpivoted_TH['Offline_source']

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_TH = unpivoted_TH[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'Offline_source_cleaned' column only
unpivoted_TH['Offline_source_cleaned'] = unpivoted_TH['Offline_source_cleaned'].str.replace(
    ":TH -  And which of these types of outdoor advertising have you seen recently for these brands?", 
    "", 
    regex=False
)

print(unpivoted_TH.head())

   Response ID                                     Offline_source  \
0        78252  Large Roadside Posters/ billboards:SAMSUNG:TH ...   
1        78269  Large Roadside Posters/ billboards:SAMSUNG:TH ...   
2        78313  Large Roadside Posters/ billboards:SAMSUNG:TH ...   
3        78345  Large Roadside Posters/ billboards:SAMSUNG:TH ...   
4        78363  Large Roadside Posters/ billboards:SAMSUNG:TH ...   

                       Offline_source_cleaned  Y/N  
0  Large Roadside Posters/ billboards:SAMSUNG  NaN  
1  Large Roadside Posters/ billboards:SAMSUNG  1.0  
2  Large Roadside Posters/ billboards:SAMSUNG  NaN  
3  Large Roadside Posters/ billboards:SAMSUNG  1.0  
4  Large Roadside Posters/ billboards:SAMSUNG  NaN  


In [69]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Offline_source_cleaned'
for entry in unpivoted_TH['Offline_source_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [70]:
# Drop the original 'Offline_source_cleaned' column
unpivoted_TH = unpivoted_TH.drop(columns=['Offline_source_cleaned'])

# Rename 'Ad_Type' to 'Offline_source_cleaned'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'Offline_source_cleaned'})

# Rearrange columns to place 'Offline_source_cleaned' next to 'Offline_source'
unpivoted_TH = unpivoted_TH[['Response ID', 'Offline_source', 'Offline_source_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_TH['Y/N'] = np.where(unpivoted_TH['Y/N'].isna(), 0, unpivoted_TH['Y/N'])

In [71]:
unpivoted_TH.head()

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78252,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
1,78269,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,1.0,SAMSUNG
2,78313,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,0.0,SAMSUNG
3,78345,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,1.0,SAMSUNG
4,78363,Large Roadside Posters/ billboards:SAMSUNG:TH ...,Large Roadside Posters/ billboards,0.0,SAMSUNG


In [72]:
count = unpivoted_TH['Y/N'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     29370
1.0       749
2.0       717
3.0       609
5.0       584
6.0       562
4.0       503
99.0       23
Name: count, dtype: int64


Total number of rows: 33117


### Concat and save them in another csv file

In [73]:
# Concatenate the melted DataFrames
combined_q17 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)

# combined_q17.dropna(subset=['Response ID'], inplace=True)

combined_q17 = combined_q17.sort_values(by='Response ID', ascending=True)

# combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q17.xlsx', float_format='%.0f', index=False)
# combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q17.xlsx', float_format='%.0f', index=False)
# combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q17.xlsx', float_format='%.0f', index=False)
combined_q17.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q17.xlsx', float_format='%.0f', index=False)

combined_q17.head(2)

,Response ID,Offline_source,Offline_source_cleaned,Y/N,Brand
0,78230,Large Roadside Posters/ billboards:BOSCH:GM - ...,Large Roadside Posters/ billboards,0.0,BOSCH
29700,78230,Large Roadside Posters/ billboards:ROWENTA:GM ...,Large Roadside Posters/ billboards,0.0,ROWENTA


In [74]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q17['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [75]:
combined_q17.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291879 entries, 0 to 180134
Data columns (total 5 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Response ID             291879 non-null  int64  
 1   Offline_source          291879 non-null  object 
 2   Offline_source_cleaned  291879 non-null  object 
 3   Y/N                     291879 non-null  float64
 4   Brand                   291879 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 13.4+ MB


In [76]:
count = combined_q17['Brand'].value_counts()
print(count)

Brand
BOSCH            24688
SAMSUNG          24688
LG               21902
DYSON            21889
ELECTROLUX       19144
MIELE            19095
AEG              18695
PHILIPS          16351
HAIER            13530
SIEMENS          13520
WHIRLPOOL         8386
BEKO              8358
HITACHI           5593
PANASONIC         5593
HOTPOINT          5558
SHARP             5193
BAUKNECHT         5160
NEFF              4356
AMICA             2800
TOSHIBA           2800
AQUA              2800
TEFAL             2793
MITSUBISHI        2793
LIEBHERR          2786
SCHULTHESS        2786
V ZUG             2786
FISHER&PAYKEL     2779
SMEG              2779
HISENSE           2779
WESTINGHOUSE      2779
ROWENTA           2772
ZANUSSI           2772
CYLINDA           2400
MIDEA             2400
XIAOMI            2400
SHARK             2376
ROBOROCK          1600
Name: count, dtype: int64


In [77]:
count = combined_q17['Y/N'].value_counts()
print(count)

Y/N
0.0     281752
1.0       2006
2.0       1933
3.0       1679
5.0       1636
6.0       1429
4.0       1244
99.0       200
Name: count, dtype: int64
